In [19]:
import pandas as pd
import matplotlib.pyplot as plt

from src.config import DEFAULT_DATA_FOLDER
from src.data.utils import save_as_excel
from src.data.preprocessing import preprocess_text

In [20]:
programming_posts = pd.read_excel(f"{DEFAULT_DATA_FOLDER}/processed/programming_posts_details.xlsx")
programming_posts_answers = pd.read_excel(f"{DEFAULT_DATA_FOLDER}/processed/programming_posts_answers_details.xlsx")

In [21]:
programming_posts =  programming_posts.drop_duplicates()
programming_posts.head(10)

,id,title,body,is_answered,view_count,accepted_answer_id,answer_count,score,tags
0,5108399,Yahoo Finance All Currencies quote API Documen...,"<p>I've being using this feed for a long time,...",True,132033,5131228.0,8,38,"json,currency,yahoo-finance"
1,17014584,How to create a delayed queue in RabbitMQ?,<p>What is the easiest way to create a delay (...,True,38548,17014585.0,6,52,"python,queue,rabbitmq,delay,pika"
2,8586206,what is stub on the &quot;server&quot; and wha...,<p>What does <em>stub</em> do on the server si...,True,50570,NaN,7,26,"java,networking,network-programming,rmi"
3,1626423,Inspect XML created by PHP SoapClient call bef...,<p>The question:\nIs there a way to view the X...,True,42188,1626572.0,3,45,"php,xml,soap,soap-client"
4,20872300,Java EE 7: How-to inject an EJB into a WebSock...,<p>To sum up my failing project: My <code>@Ser...,True,9670,25802434.0,2,20,"dependency-injection,websocket,ejb,java-ee-7,t..."
5,10693955,Stubbing or Mocking ASP.NET Web API HttpClient,<p>I am using the new Web API bits in a projec...,True,23439,12247745.0,3,28,"c#,unit-testing,dotnet-httpclient,rhino-mocks"
6,12428664,Automatically processing the Amazon SES notifi...,<p>We are using AWS SES for sending mails. Ama...,True,10124,NaN,5,13,"amazon-web-services,email,notifications,email-..."
7,656934,ActiveX component can&#39;t create object,<p>I have just installed a third party app on ...,True,271292,660081.0,12,36,"dll,vbscript,activex,windows-server-2008,dllre..."
8,1310520,C# Amazon Product Advertising API,"<p>As of August 15, Amazon made it compulsory ...",True,7025,1310844.0,2,10,"c#,soap,amazon-product-api"
9,11155176,Removing headers from the response,<p>I need to cloak certain headers generated b...,True,9677,11155985.0,4,13,"asp.net-web-api,wcf-web-api"


In [22]:
programming_posts_answers = programming_posts_answers.drop_duplicates()
programming_posts_answers.head(10)

,question_id,answer_id,body,is_accepted,score
0,5108399,5131228,"<p>From the research that I've done, there doe...",True,23
1,17014584,17014585,<p>I found this extremely useful when developi...,True,104
2,8586206,29343313,<p>look at the followed picture:</p>\n\n<p><im...,False,25
3,1626423,1626572,<p><strong>Upfront remark:</strong> In order t...,True,87
4,20872300,25802434,<p>(just restating what I wrote into comment t...,True,3
5,10693955,12247745,<p>I use Moq and I can stub out the HttpClient...,True,11
6,12428664,13636182,<p>I find that directly subscribing to SNS usi...,False,10
7,656934,660081,<p>It turns out to get this application workin...,True,51
8,1310520,1310844,"<p>Check out <a href=""http://flyingpies.wordpr...",True,13
9,11155176,11155985,<p>The problem is each one is added at a diffe...,True,11


In [24]:
merge1 = pd.merge(programming_posts, programming_posts_answers, how='left', left_on='id', right_on='question_id')

merge1['text'] = merge1['title'] + ' ' + merge1['body_x'].fillna(' ') + ' ' + merge1['body_y'].fillna(' ')

merge1['text'] = merge1['text'].fillna(' ').apply(preprocess_text)

merge1['clean_text'] = merge1['text'].apply(preprocess_text)
merge1['total_tokens'] = merge1['clean_text'].apply(lambda x: len(x.split()))
final_results = merge1[['id', 'clean_text', 'is_answered', 'is_accepted', 'tags', 'total_tokens']].copy()
 
final_results = final_results.drop_duplicates()
final_results.head(10)

,id,clean_text,is_answered,is_accepted,tags,total_tokens
0,5108399,yahoo finance currency quote api documentation...,True,True,"json,currency,yahoo-finance",135
1,17014584,create delayed queue rabbitmq easy way create ...,True,True,"python,queue,rabbitmq,delay,pika",296
2,8586206,stub server skeleton mean stub server side ske...,True,False,"java,networking,network-programming,rmi",85
3,1626423,inspect xml create php soapclient call beforew...,True,True,"php,xml,soap,soap-client",215
4,20872300,java ee how-to inject ejb websocket serverendp...,True,True,"dependency-injection,websocket,ejb,java-ee-7,t...",325
5,10693955,stub mock aspnet web api httpclient use new we...,True,True,"c#,unit-testing,dotnet-httpclient,rhino-mocks",168
6,12428664,automatically processing amazon notification b...,True,False,"amazon-web-services,email,notifications,email-...",114
7,656934,activex component create object instal third p...,True,True,"dll,vbscript,activex,windows-server-2008,dllre...",87
8,1310520,c amazon product advertising api august amazon...,True,True,"c#,soap,amazon-product-api",107
9,11155176,remove header response need cloak certain head...,True,True,"asp.net-web-api,wcf-web-api",97


In [25]:
print()
print("Total posts: ", len(programming_posts))
print("Total posts found: ", len(programming_posts))
print("Total answers found: ", len(programming_posts_answers))
print("Total posts with accepted answers: ", len(final_results[final_results['is_accepted'] == True]))
print("Total posts with answers: ", len(final_results))


Total posts:  1620
Total posts found:  1620
Total answers found:  1615
Total posts with accepted answers:  1188
Total posts with answers:  1619


In [26]:
save_as_excel(final_results, f"{DEFAULT_DATA_FOLDER}/output/programming_posts_results.xlsx")
print("Completed")

Completed
